In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

    Takeaways:
        - replace \n or \r sequences with single \n
        - use language-specific tokenizer

# BOW stats
    # TODO: add actual bow stats

In [11]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [12]:
data = data[data["synopsis"] != ""]

In [13]:
import regex as re

data["synopsis"] = data["synopsis"].apply(
    lambda x: re.sub("\s*\n\s*", "\n", x).strip()
)

In [14]:
# data["min_ord"] = data["synopsis"].progress_apply(
#     lambda s: min([ord(char) for char in s])
# )

# data["max_ord"] = data["synopsis"].progress_apply(
#     lambda s: max([ord(char) for char in s])
# )

data["mean_ord"] = data["synopsis"].progress_apply(
    lambda s: np.mean([ord(char) for char in s])
)

# data["median_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.median([ord(char) for char in s])
# )

# data["mode_ord"] = data["synopsis"].progress_apply(
#     lambda s: Counter([ord(char) for char in s]).most_common()[0][0]
# )

  0%|          | 0/18374 [00:00<?, ?it/s]

In [12]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(
    lambda x: len(word_tokenize(x))
)
data["sent_total"] = data["synopsis"].progress_apply(
    lambda x: len([sent for line in x.split("\n") for sent in  sent_tokenize(line) if line.strip()])
)

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

In [27]:
# data["lines_total"] = data["synopsis"].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/19393 [00:00<?, ?it/s]

In [5]:
data["token_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in word_tokenize(x)]) if x.strip() else 0
)

data["sent_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in sent_tokenize(x)]) if x.strip() else 0
)

data["line_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in x.split("\n")]) if x.strip() else 0
)

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [16]:
data["sent_char_ratio"] = [t.sent_total/t.char_total for t in data.itertuples()]
data["sent_token_ratio"] = [t.sent_total/t.token_total for t in data.itertuples()]
data["token_char_ratio"] = [t.token_total/t.char_total for t in data.itertuples()]
data["token_sent_ratio"] = [t.token_total/t.sent_total for t in data.itertuples()]
data["char_sent_ratio"] = [t.char_total/t.sent_total for t in data.itertuples()]
data["char_token_ratio"] = [t.char_total/t.token_total for t in data.itertuples()]

In [13]:
data.describe().astype(int)

,char_total,token_total,sent_total
count,18374,18374,18374
mean,519,105,5
std,348,70,3
min,1,1,1
25%,262,53,3
50%,454,92,5
75%,700,141,7
max,5799,1201,79


In [15]:
stats = data.drop(columns=["href", "synopsis"]) #.drop(columns=["line_max_char_len"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [16]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [17]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)

data["pred"] = preds
data["negative_outlier_factor_"] = clf.negative_outlier_factor_

In [18]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [19]:
data[data["pred"] == -1].sort_values(by="negative_outlier_factor_"
                                    )[['href', 'synopsis', 'pred', 'negative_outlier_factor_']].head(40)

,href,synopsis,pred,negative_outlier_factor_
9387,/56367-test-content-season-4,押イおッ人高ラヘ致24格会ケロヲク放文イメラシ倍極ろぼるけ島判険おが会影のたト床用じフ索手工ふ決39客芝2会促縮繰しー。議づだーめ景能オマ囲索趣こイが車74客芝6交いかへづ談情そゆあん明臨ヲホセサ博位カフク逃転地掘ークゃ年柏沼烈翌け。樹ネエ未著艦ぎう請報ア時阪便テコス学医テニ限物うてを嘉一るなむあ豪権ノマ慢活ーひや頑受やきイょ制団ぴゅめ捕活そクぱ泡死スチメ法乏咋塞墨はラい。\n扱ドみば質削...,-1,-21.646124
14223,/41937-2012-big-bang-alive-tour-bigshow,2012 Big Bang Alive Tour: Bigshow\nnative\t2012 빅뱅 얼라이브 투어: 빅쇼,-1,-6.901991
2401,/736603-a-very-lucky-day,"~~ Adapted from the webtoon ""Unsuojinnal"" (운수 오진 날) by Aporia (아포리아).",-1,-4.590387
3938,/733521-blueming-season-2,"~~ Adapted from the webtoon ""Who Can Define Popularity?"" (인기는 무언가 잘못되었다?) by Tak Bon (拓本).",-1,-4.459703
4145,/705589-taxi-driver-2,"~~ Adapted from the webtoon ""The Deluxe Taxi"" (모범택시) by Carlos (까를로스) and Keukeu Jae Jin (크크재진).",-1,-4.283625
13447,/26244-game-not-over,N/A,-1,-3.662925
18110,/32578-the-dark-lord-season-2,N/A,-1,-3.662925
18896,/11956-messiah-shikkoku-no-shou,N/A,-1,-3.662925
9567,/27070-facebook-caf,N/A,-1,-3.662925
16662,/28099-sword-chaos,N/A,-1,-3.662925
